# Data Warehouse Design – Star Schema

## a. Design a star schema for the e-commerce data

refer to ecommerce_star_schema_v1.8.drawio in /docs

![Star Schema](../docs/ecommerce_star_schema_v1.8.png)


#b. Create Dimension and Fact Tables

FactSales

| Column                           | Description                     |
| -------------------------------- | ------------------------------- |
| order\_id, order\_item\_id (Composite PK)                   | Order ID, Item index in the order                        |
| product\_id (FK)                 | Product sold                    |
| seller\_id (FK)                  | Seller                          |
| customer\_id (FK)                | Customer                        |
| price                            | Product price                   |
| freight\_value                   | Freight cost                    |
| payment\_value                   | Total paid (from payment table) |
| payment\_type                    | Payment method                  |
| review\_score                    | Score from review               |
| order\_status                    | Delivered, shipped, etc.        |
| order\_purchase\_timestamp       | Purchase date                   |
| order\_delivered\_customer\_date | Delivery date                   |
| order\_estimated\_delivery\_date | Estimated delivery              |


DimCustomer

| Column                           | Description        |
| -------------------------------- | ------------------ |
| customer\_id (PK)                | Unique customer ID |
| customer\_unique\_id             | Logical person ID  |
| customer\_zip\_code\_prefix (FK) | ZIP prefix         |
| customer\_city                   | City               |
| customer\_state                  | State              |


DimProduct

| Column                       | Description        |
| ---------------------------- | ------------------ |
| product\_id (PK)             | Product ID         |
| product\_category\_name      | In Portuguese      |
| product\_name\_length        | Text length        |
| product\_description\_length | Description length |
| product\_photos\_qty         | Number of photos   |
| product\_weight\_g           | Weight             |
| product\_length\_cm          | Length             |
| product\_height\_cm          | Height             |
| product\_width\_cm           | Width              |


DimSeller

| Column                         | Description |
| ------------------------------ | ----------- |
| seller\_id (PK)                | Seller ID   |
| seller\_zip\_code\_prefix (FK) | ZIP prefix  |
| seller\_city                   | City        |
| seller\_state                  | State       |


DimDate

| Column        | Description  |
| ------------- | ------------ |
| date (PK)     | Date value   |
| year          | Year         |
| month         | Month        |
| day           | Day          |
| week          | ISO week     |
| weekday\_name | e.g., Monday |
| is\_weekend   | True/False   |


DimGeolocation

| Column                 | Description |
| ---------------------- | ----------- |
| zip\_code\_prefix (PK) | ZIP prefix  |
| latitude               | Avg lat     |
| longitude              | Avg long    |
| city                   | City        |
| state                  | State       |

# Fact Table - fact_sales_corrected

Central fact table containing order metrics, foreign keys to dimensions

In [ ]:
WITH order_items AS (
    SELECT * FROM {{ source('BET_Team3', 'order_items_corrected') }}
),

orders AS (
    SELECT * FROM {{ source('BET_Team3', 'orders_corrected') }}
),

products AS (
    SELECT * FROM {{ source('BET_Team3', 'products_corrected') }}
),

sellers AS (
    SELECT * FROM {{ source('BET_Team3', 'sellers') }}
),

payments AS (
    SELECT * FROM {{ source('BET_Team3', 'order_payments') }}
),

reviews AS (
    SELECT * FROM {{ source('BET_Team3', 'order_reviews') }}
),

joined_data AS (
    SELECT
        o.order_id,
        o.order_item_id,
        ord.customer_id,
        o.product_id,
        p.product_category_name,
        s.seller_id,
        s.seller_city,
        s.seller_state,
        ord.order_status,
        ord.order_purchase_timestamp,
        ord.order_approved_at,
        ord.order_delivered_carrier_date,
        ord.order_delivered_customer_date,
        ord.order_estimated_delivery_date,
        o.price,
        o.freight_value,
        pay.payment_type,
        pay.payment_value,
        pay.payment_sequential,
        r.review_score,
        ROW_NUMBER() OVER (
            PARTITION BY o.order_id, o.order_item_id
            ORDER BY ord.order_purchase_timestamp DESC
        ) AS row_num
    FROM order_items o
    JOIN orders         ord ON o.order_id = ord.order_id
    JOIN products       p   ON o.product_id = p.product_id
    JOIN sellers        s   ON o.seller_id = s.seller_id
    LEFT JOIN payments  pay ON pay.order_id = o.order_id
    LEFT JOIN reviews   r   ON r.order_id = o.order_id
)

SELECT *
FROM joined_data
WHERE row_num = 1

## dim_products

Product details

In [ ]:
SELECT
  product_id,
  product_category_name,
  product_name_length,
  product_description_length,
  product_photos_qty,
  product_weight_g,
  product_length_cm,
  product_height_cm,
  product_width_cm
FROM {{ source('BET_Team3', 'products_corrected') }}

# dim_order_reviews

Review scores and feedback data linked to orders

In [ ]:
WITH base AS (
    SELECT
        review_id,
        order_id,
        CONCAT(order_id, '_', review_id) AS order_id_review_id,
        review_score,
        review_comment_title,
        review_comment_message,
        review_creation_date,
        review_answer_timestamp
    FROM {{ source('BET_Team3', 'order_reviews') }}
)

SELECT *
FROM base


# dim_payments

Payment method and transaction details

In [ ]:
SELECT
  order_id,
  payment_type,
  payment_installments
FROM {{ source('BET_Team3', 'order_payments') }}


# dim_customers

Customer details and address information

In [ ]:
SELECT
    customer_id,
    customer_unique_id,
    customer_zip_code_prefix, 
    customer_city,
    customer_state
FROM {{ source('BET_Team3', 'customers') }}


# dim_sellers

Seller details and address information

In [ ]:
SELECT
  seller_id,
  seller_zip_code_prefix,
  seller_city,
  seller_state
FROM {{ source('BET_Team3', 'sellers') }}


# dim_geolocation

Geolocation Data

In [ ]:
WITH raw_geo AS (
    SELECT DISTINCT
        LPAD(CAST(geolocation_zip_code_prefix AS STRING), 5, '0') AS geolocation_zip_code_prefix,
        geolocation_lat,
        geolocation_lng,
        LOWER(TRIM(geolocation_city)) AS geolocation_city,
        UPPER(TRIM(geolocation_state)) AS geolocation_state
    FROM {{ source('BET_Team3', 'geolocation_version1') }}
),

with_surrogate AS (
    SELECT
        ROW_NUMBER() OVER () AS surrogate_key,
        *
    FROM raw_geo
)

SELECT * FROM with_surrogate

# dim_date

Date dimension for temporal analysis

In [ ]:
with raw_dates as (
    select distinct
        date(order_purchase_timestamp) as date_id
    from {{ source('BET_Team3', 'orders_corrected') }}
),

final as (
    select
        date_id,
        extract(year from date_id) as year,
        extract(month from date_id) as month,
        extract(day from date_id) as day,
        format_date('%A', date_id) as day_name,
        format_date('%B', date_id) as month_name
    from raw_dates
)

select * from final

# dim_time

Time dimension for temporal analysis

In [ ]:
with times as (
    select distinct cast(order_purchase_timestamp as timestamp) as time
    from {{ source('BET_Team3', 'orders_corrected') }}
    union distinct
    select distinct cast(order_approved_at as timestamp) as time
    from {{ source('BET_Team3', 'orders_corrected') }}
    union distinct
    select distinct cast(order_delivered_carrier_date as timestamp) as time
    from {{ source('BET_Team3', 'orders_corrected') }}
    union distinct
    select distinct cast(order_delivered_customer_date as timestamp) as time
    from {{ source('BET_Team3', 'orders_corrected') }}
    union distinct
    select distinct cast(order_estimated_delivery_date as timestamp) as time
    from {{ source('BET_Team3', 'orders_corrected') }}
),
final as (
    select
        time as time_id,
        extract(hour from time) as hour,
        extract(minute from time) as minute,
        format_time('%H:%M', cast(time as TIME)) as time_hhmm
    from times
    where time is not null
)
select * from final


